<a href="https://colab.research.google.com/github/mariemtouihri/GRAM-Metric/blob/main/Closeness_Centrality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Import libraries and data

In [ ]:
import numpy as np
import networkx as nx
from scipy.stats import spearmanr, kendalltau
import matplotlib.pyplot as plt
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
  Load all matrices from my drive in a dictionary (There are 8 Groups of matrices,
   each time we will work with a specific group in all cells and then repeat all
    of it to the next group)

"""

# Read the adjacency matrix
loaded_data_dict = {}
noises = [i for i in range(11)]
j=2 # To select G1 and so on ..
for noise in noises:
  noise *=10
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/Copie de matrices_noise_G{j}_{noise}%.npy'
  loaded_data = np.load(file_path)
  loaded_data_dict[noise] = loaded_data

print(f"Matrices G{j} are loaded !")

In [ ]:
len(loaded_data_dict[0])

1100

### Define needed functions

In [ ]:
def closeness (matrices):

    closeness_values = []
    for i in range(len(matrices)):
      matrix = matrices[i]
      G = nx.DiGraph(matrix)
      closeness_v = nx.closeness_centrality(G, distance='weight')

      closeness_list = list(closeness_v.values())
      closeness_values.extend(closeness_list)

    return closeness_values

In [ ]:
# This metric works on node level ! (it is different from metrics that work on graph level)
def calculate_correlations(original_values, modified_values, j, x):
  p = []
  sp = []
  ktau = []
  corr_dict = {}
  for i in range(0, len(original_values), 90):
    # Calculate Pearson Correlation
    p_correlation = np.corrcoef(original_values[i:i+90], modified_values[i:i+90])[0, 1]
    p.append(p_correlation)

    # Calculate Spearman correlation
    sp_correlation, _ = spearmanr(original_values[i:i+90], modified_values[i:i+90])
    sp.append(sp_correlation)

    # Calculate Kendall Tau correlation
    ktau_correlation, p_value = kendalltau(original_values[i:i+90], modified_values[i:i+90])
    ktau.append(ktau_correlation)


  # Store correlation results to a json file for each matrix
  corr_dict['pearson_values'] = p
  corr_dict['spearman_values'] = sp
  corr_dict['ktau_values'] = ktau


  # Define the file path for the new JSON file
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/closeness/closeness_all_correlations_matrices_G{j}_{x}%.json'

  # Serialize and save the dictionary to the new file
  with open(file_path, 'w') as json_file:
      json.dump(corr_dict, json_file)

  print(f"Results saved to {file_path}")




  # Calculate avg of correlations of all matrices
  pearson_value = np.mean(p, axis=0)
  spearman_value = np.mean(sp, axis=0)
  ktau_value = np.mean(ktau, axis=0)

  return pearson_value, spearman_value, ktau_value

### Start calculations and save them to files


In [ ]:
loaded_data_dict = {}
noises = [i for i in range(0,110,10)]

# To select G1 and so on ..
for j in range(1,8):

  for noise in noises:
    file_path = f'/content/drive/MyDrive/Colab_Notebooks/Copie de matrices_noise_G{j}_{noise}%.npy'
    loaded_data = np.load(file_path)
    loaded_data_dict[noise] = loaded_data

  print(f"Matrices G{j} are loaded !")






  closeness_dict = {} # to store results in a dictionary

  for percent, loaded_data in loaded_data_dict.items():
      closeness_dict[percent] = closeness(loaded_data)



  # store results in an .npy file

  dict_values = list(closeness_dict.values())
  dict_array = np.array(dict_values)

  file_path = f'/content/drive/MyDrive/Colab_Notebooks/closeness/closeness_values_matrices_G{j}.npy'

  np.save(file_path, dict_array)
  print(f"Values G{j} are saved")

In [ ]:
j=1
file_path = f'/content/drive/MyDrive/Colab_Notebooks/closeness/closeness_values_matrices_G{j}.npy'
data = np.load(file_path)
noises = [ i for i in range(0,110,10)]
closeness_dict= {}
i=0
for noise in noises:

    closeness_dict[noise] = data[i]
    i+=1

print(len(closeness_dict[0]))

In [ ]:
noises = [ i for i in range(0,110,10)]

for j in  range(1,9,1):
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/closeness/closeness_values_matrices_G{j}.npy'
  data = np.load(file_path)
  hub_auth_dict= {}
  i=0
  for noise in noises:
      hub_auth_dict[noise] = data[i]
      i+=1

  print(len(hub_auth_dict))


  pearson_values = [1]
  spearman_values = [1]
  ktau_values = [1]

  for i in range(10,110,10):
    pearson_value, spearman_value, ktau_value = calculate_correlations(hub_auth_dict[0], hub_auth_dict[i], j, i)
    pearson_values.append(pearson_value)
    spearman_values.append(spearman_value)
    ktau_values.append(ktau_value)



  # Store correlation results to a json file for each G

  corr_dict = {}
  corr_dict['pearson_values'] = pearson_values
  corr_dict['spearman_values'] = spearman_values
  corr_dict['ktau_values'] = ktau_values


  # Define the file path for the new JSON file
  file_path = f'/content/drive/MyDrive/Colab_Notebooks/closeness/closeness_correlations_matrices_G{j}.json'

  # Serialize and save the dictionary to the new file
  with open(file_path, 'w') as json_file:
      json.dump(corr_dict, json_file)

  print(f"Results saved to {file_path}")


In [ ]:
pearson_values = [1]
spearman_values = [1]
ktau_values = [1]

for i in range(10,110,10):
  pearson_value, spearman_value, ktau_value = calculate_correlations(closeness_dict[0], closeness_dict[i], j, i)
  pearson_values.append(pearson_value)
  spearman_values.append(spearman_value)
  ktau_values.append(ktau_value)



# Store correlation results to a json file for each G

corr_dict = {}
corr_dict['pearson_values'] = pearson_values
corr_dict['spearman_values'] = spearman_values
corr_dict['ktau_values'] = ktau_values


# Define the file path for the new JSON file
file_path = f'/content/drive/MyDrive/Colab_Notebooks/closeness/closeness_correlations_matrices_G{j}.json'

# Serialize and save the dictionary to the new file
with open(file_path, 'w') as json_file:
    json.dump(corr_dict, json_file)

print(f"Results saved to {file_path}")

In [ ]:
j=5
file_path = f'/content/drive/MyDrive/Colab_Notebooks/closeness/closeness_values_matrices_G{j}.npy'
data = np.load(file_path)
noises = [ i for i in range(0,110,10)]
closeness_dict= {}
i=0
for noise in noises:

    closeness_dict[noise] = data[i]
    i+=1

print(len(closeness_dict))

11


In [ ]:
# Plot results
modification_rates = [f"{i}%" for i in range(0,110,10)]

plt.figure(figsize=(8, 6))

plt.plot(modification_rates, pearson_values, marker='o', label= "Pearson")
plt.plot(modification_rates, spearman_values, marker='o', label= "Spearman")
plt.plot(modification_rates, ktau_values, marker='o', label= "Kendall Tau")

plt.xlabel("Modification Percentage")
plt.ylabel("Correlation")
plt.title("Correlation of Closeness Centrality (Original vs Modified)")
plt.legend()
plt.grid(True)
plt.show()